In [4]:
# Instalar bibliotecas ( necessário)
!pip install mlflow evidently scikit-learn pandas numpy matplotlib seaborn papermill

# Importar pacotes principais
# Pré-requisitos:
# pip install pandas scikit-learn pyarrow mlflow dvc

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pickle
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from evidently.report import Report
from evidently.metrics import DataDriftTable
import pickle


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Definindo um novo diretório para armazenar os logs dos experimentos
caminho_logs = "E:/Mestrado UFCG/Semestre 2024.2/Dados/Aplicando_para_dissertacao/mlflow_logs"

# Criar o diretório se não existir
os.makedirs(caminho_logs, exist_ok=True)

# Convertendo o caminho para o formato correto de URI
caminho_logs_uri = f"file:///{os.path.abspath(caminho_logs)}"

# Configurando o MLflow para usar esse diretório
mlflow.set_tracking_uri(caminho_logs_uri)

# Criando ou definindo o experimento
mlflow.set_experiment("Evasao_UFCG")

# Exibir a configuração para garantir que está correto
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

Tracking URI: file:///E:\Mestrado UFCG\Semestre 2024.2\Dados\Aplicando_para_dissertacao\mlflow_logs


### Carregar tabelas

In [6]:
# Definindo caminhos
caminho_base = "E:/Mestrado UFCG/Semestre 2024.2/Dados/Tabelas_0/"
tables = ["alunos", "matriculas"]

# Iniciar Experimento MLflow
mlflow.set_experiment("Carregamento de Dados UFCG")

with mlflow.start_run():
    # Log do início do carregamento usando métrica em vez de parâmetro fixo
    mlflow.log_metric("status_iniciando", 1)

    dados = {}
    
    for table in tables:
        caminho = os.path.join(caminho_base, f"{table}.csv")
        try:
            # Detectar delimitador correto
            with open(caminho, "r", encoding="utf-8") as f:
                primeira_linha = f.readline()
                if ";" in primeira_linha:
                    delimitador = ";"
                elif "," in primeira_linha:
                    delimitador = ","
                elif "\t" in primeira_linha:
                    delimitador = "\t"
                else:
                    delimitador = ","  # Padrão caso não detecte

            # Carregar CSV com delimitador correto
            df = pd.read_csv(caminho, delimiter=delimitador, encoding="utf-8")
            
            # Verifica se carregou corretamente
            if df.shape[1] == 1:
                print(f" Atenção: {table} foi carregada com apenas 1 coluna! Pode haver erro no delimitador.")
            
            dados[table] = df
            print(f"{table} carregada com sucesso! ({df.shape[0]} linhas, {df.shape[1]} colunas)")
        except Exception as e:
            print(f"Erro ao carregar {table}: {e}")
            mlflow.log_param(f"erro_{table}", str(e))  # Log do erro no MLflow

    # Usar métrica ao invés de parâmetro para evitar erro no MLflow
    mlflow.log_metric("status_carregamento_finalizado", 1)

    print(" Carregamento finalizado.")


alunos carregada com sucesso! (3761 linhas, 22 colunas)
matriculas carregada com sucesso! (182660 linhas, 10 colunas)
 Carregamento finalizado.


### REmoção dados duplicados

In [7]:
import pandas as pd

# Caminho base onde as tabelas estão armazenadas
caminho_base = "E:/Mestrado UFCG/Semestre 2024.2/Dados/Tabelas_0/"

# Lista com os nomes das tabelas
tables = ["alunos", "matriculas"]

# Carregar e exibir as colunas de cada tabela
for tabela in tables:
    # Carregar a tabela especificando o separador ';'
    tabela_df = pd.read_csv(caminho_base + tabela + ".csv", sep=';', on_bad_lines='skip')
    
    # Se a tabela for "matriculas", aplicar o filtro de ano e semestre
    if tabela == "matriculas":
        # Garantir que a coluna 'TERMO' esteja em formato adequado (se necessário)
        tabela_df['TERMO'] = tabela_df['TERMO'].astype(str)
        
        # Filtrar os registros entre 2002.1 e 2023.2
        tabela_df = tabela_df[tabela_df['TERMO'].between('2002.1', '2023.2')]
        
        # Remover duplicatas com base na coluna 'MATRICULA' (ou qualquer outra que identifique um aluno único)
        tabela_df = tabela_df.drop_duplicates(subset=['MATRICULA'], keep='first')  # Mantém a primeira ocorrência de cada matrícula
    
    # Mostrar as dimensões (linhas e colunas) da tabela após o tratamento
    print(f"\nTabela '{tabela}' tem {tabela_df.shape[0]} linhas e {tabela_df.shape[1]} colunas após tratamento de duplicação.")



Tabela 'alunos' tem 3761 linhas e 22 colunas após tratamento de duplicação.

Tabela 'matriculas' tem 3789 linhas e 10 colunas após tratamento de duplicação.


### Merge das 2 tabelas

In [8]:
# Caminho base onde as tabelas estão armazenadas
caminho_base = "E:/Mestrado UFCG/Semestre 2024.2/Dados/Tabelas_0/"

# Nomes das tabelas
tables = ["alunos", "matriculas"]

# Carregar as tabelas
alunos_df = pd.read_csv(caminho_base + "alunos.csv", sep=';', on_bad_lines='skip')
matriculas_df = pd.read_csv(caminho_base + "matriculas.csv", sep=';', on_bad_lines='skip')

# Realizar o merge das duas tabelas com base na coluna 'MATRICULA'
merged_df = pd.merge(alunos_df, matriculas_df, on='MATRICULA', suffixes=('_alunos', '_matriculas'))

# Comparar a coluna 'NOME' nas duas tabelas
merged_df['NOME_iguais'] = merged_df['NOME_alunos'] == merged_df['NOME_matriculas']

# Se os nomes forem iguais, remover a coluna 'NOME_matriculas'
merged_df = merged_df[~merged_df['NOME_iguais']].drop(columns=['NOME_matriculas', 'NOME_iguais'])

# Exibir o resultado final
print("Tabela final após o merge e remoção de NOME duplicado:")
print(merged_df.head())

Tabela final após o merge e remoção de NOME duplicado:
   MATRICULA    ID_CIDADAO                NOME_alunos  IDADE E-MAIL  \
0  102210002  5.175670e+09  ELISMAEL GUIMARAES MENINO     43    NaN   
1  102210002  5.175670e+09  ELISMAEL GUIMARAES MENINO     43    NaN   
2  102210002  5.175670e+09  ELISMAEL GUIMARAES MENINO     43    NaN   
3  102210002  5.175670e+09  ELISMAEL GUIMARAES MENINO     43    NaN   
4  102210002  5.175670e+09  ELISMAEL GUIMARAES MENINO     43    NaN   

      GENERO ESTADO_CIVIL_ALUNOS NACIONALIDADE LOCAL_NASCIMENTO    ESTADO  \
0  MASCULINO            SOLTEIRO    BRASILEIRA      PIANC? - PB  GRADUADO   
1  MASCULINO            SOLTEIRO    BRASILEIRA      PIANC? - PB  GRADUADO   
2  MASCULINO            SOLTEIRO    BRASILEIRA      PIANC? - PB  GRADUADO   
3  MASCULINO            SOLTEIRO    BRASILEIRA      PIANC? - PB  GRADUADO   
4  MASCULINO            SOLTEIRO    BRASILEIRA      PIANC? - PB  GRADUADO   

   ...  EX_ALUNOS ALUNOS_INATIVOS CODIGO_DISCIPLINA  CR

In [9]:
# Renomear a coluna 'NOME_alunos' para 'NOME'
merged_df = merged_df.rename(columns={'NOME_alunos': 'NOME'})

# Visualizar o número de linhas e colunas
num_linhas, num_colunas = merged_df.shape

# Exibir as informações
print(f"A tabela resultante possui {num_linhas} linhas e {num_colunas} colunas.")
print("Primeiras 5 linhas da tabela:")
print(merged_df.head())

A tabela resultante possui 130559 linhas e 30 colunas.
Primeiras 5 linhas da tabela:
   MATRICULA    ID_CIDADAO                       NOME  IDADE E-MAIL  \
0  102210002  5.175670e+09  ELISMAEL GUIMARAES MENINO     43    NaN   
1  102210002  5.175670e+09  ELISMAEL GUIMARAES MENINO     43    NaN   
2  102210002  5.175670e+09  ELISMAEL GUIMARAES MENINO     43    NaN   
3  102210002  5.175670e+09  ELISMAEL GUIMARAES MENINO     43    NaN   
4  102210002  5.175670e+09  ELISMAEL GUIMARAES MENINO     43    NaN   

      GENERO ESTADO_CIVIL_ALUNOS NACIONALIDADE LOCAL_NASCIMENTO    ESTADO  \
0  MASCULINO            SOLTEIRO    BRASILEIRA      PIANC? - PB  GRADUADO   
1  MASCULINO            SOLTEIRO    BRASILEIRA      PIANC? - PB  GRADUADO   
2  MASCULINO            SOLTEIRO    BRASILEIRA      PIANC? - PB  GRADUADO   
3  MASCULINO            SOLTEIRO    BRASILEIRA      PIANC? - PB  GRADUADO   
4  MASCULINO            SOLTEIRO    BRASILEIRA      PIANC? - PB  GRADUADO   

   ...  EX_ALUNOS ALUNOS_

In [10]:
# Remover duplicatas baseadas na coluna 'MATRICULA' ou em todas as colunas
merged_df = merged_df.drop_duplicates(subset='MATRICULA', keep='first')

# Ou, se preferir remover todas as duplicatas baseadas em todas as colunas:
# merged_df = merged_df.drop_duplicates(keep='first')

# Visualizar o número de linhas e colunas após a remoção das duplicatas
num_linhas, num_colunas = merged_df.shape

# Exibir as informações
print(f"A tabela resultante possui {num_linhas} linhas e {num_colunas} colunas após remoção de duplicatas.")
print("Primeiras 5 linhas da tabela:")
print(merged_df.head())

A tabela resultante possui 3508 linhas e 30 colunas após remoção de duplicatas.
Primeiras 5 linhas da tabela:
     MATRICULA    ID_CIDADAO                             NOME  IDADE E-MAIL  \
0    102210002  5.175670e+09        ELISMAEL GUIMARAES MENINO     43    NaN   
90   102210003           NaN      EDUARDO JOSE MOREIRA COLACO     40    NaN   
119  102210004  2.454245e+08  ADILTON ANGELO SEIXAS MAGALHAES     43    NaN   
175  102210005  1.195394e+09        RICARDO MADEIRA FERNANDES     43    NaN   
235  102210006  4.220965e+09            BRUNO COITINHO ARAUJO     40    NaN   

        GENERO ESTADO_CIVIL_ALUNOS NACIONALIDADE     LOCAL_NASCIMENTO  \
0    MASCULINO            SOLTEIRO    BRASILEIRA          PIANC? - PB   
90   MASCULINO            SOLTEIRO    BRASILEIRA  CAMPINA GRANDE - PB   
119  MASCULINO            SOLTEIRO    BRASILEIRA                  NaN   
175  MASCULINO            SOLTEIRO    BRASILEIRA       BOA VISTA - RR   
235  MASCULINO            SOLTEIRO    BRASILEIRA  

In [12]:
import pandas as pd

# Verificar colunas com valores ausentes em merged_df
colunas_ausentes = merged_df.columns[merged_df.isnull().any()]

if len(colunas_ausentes) > 0:
    print("\nColunas com valores ausentes em 'merged_df':")
    print(colunas_ausentes.tolist())
else:
    print("\n'merged_df' não contém colunas com valores ausentes.")



'merged_df' não contém colunas com valores ausentes.


In [13]:
# Tratamento dos valores ausentes
merged_df['ID_CIDADAO'].fillna(-1, inplace=True)
merged_df['E-MAIL'].fillna("Não informado", inplace=True)
merged_df['LOCAL_NASCIMENTO'].fillna("Desconhecido", inplace=True)
merged_df['TERMO_ESTADO'].fillna(merged_df['TERMO_ESTADO'].mode()[0], inplace=True)  # Preenche com a moda
merged_df['ANO_FORMATURA_ENSINO_MEDIO'].fillna(merged_df['ANO_FORMATURA_ENSINO_MEDIO'].median(), inplace=True)
merged_df['ID_CLASS'].fillna(-1, inplace=True)

# Verificar se ainda existem valores ausentes
print(merged_df[colunas_ausentes].isnull().sum())

Series([], dtype: float64)


C:\Users\Big Data\AppData\Local\Temp\ipykernel_19156\3168965822.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['ID_CIDADAO'].fillna(-1, inplace=True)
C:\Users\Big Data\AppData\Local\Temp\ipykernel_19156\3168965822.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

In [14]:
print(merged_df.isnull().sum().sum())  # Deve retornar 0 se tudo estiver tratado


0


### Colunas relevantes

In [15]:
# Definição das colunas relevantes
colunas_relevantes = [
    'IDADE', 'GENERO', 'ESTADO_CIVIL_ALUNOS', 'NACIONALIDADE', 'TIPO_ADMISSAO',
    'POLITICA_AFIRMATIVA', 'TIPO_ENSINO_MEDIO', 'ANO_FORMATURA_ENSINO_MEDIO',
    'CODIGO_CURSO', 'TERMO_ESTADO', 'CREDITOS', 'HORAS', 'NOTA', 'ESTATUS', 'ESTADO'
]

# Criar novo DataFrame apenas com as colunas selecionadas
df_modelo = merged_df[colunas_relevantes].copy()

# Exibir as 5 primeiras linhas para conferir
print(df_modelo.head())

     IDADE     GENERO ESTADO_CIVIL_ALUNOS NACIONALIDADE TIPO_ADMISSAO  \
0       43  MASCULINO            SOLTEIRO    BRASILEIRA    VESTIBULAR   
90      40  MASCULINO            SOLTEIRO    BRASILEIRA    VESTIBULAR   
119     43  MASCULINO            SOLTEIRO    BRASILEIRA    VESTIBULAR   
175     43  MASCULINO            SOLTEIRO    BRASILEIRA    VESTIBULAR   
235     40  MASCULINO            SOLTEIRO    BRASILEIRA    VESTIBULAR   

    POLITICA_AFIRMATIVA TIPO_ENSINO_MEDIO  ANO_FORMATURA_ENSINO_MEDIO  \
0                    A0      DESCONHECIDA                      2014.0   
90                   A0      DESCONHECIDA                      2014.0   
119                  A0      DESCONHECIDA                      2014.0   
175                  A0      DESCONHECIDA                      2014.0   
235                  A0      DESCONHECIDA                      2014.0   

     CODIGO_CURSO  TERMO_ESTADO  CREDITOS  HORAS  NOTA   ESTATUS    ESTADO  
0        14102100        2009.1         4    

In [16]:
# Remover alunos que se formaram
df_modelo = df_modelo[df_modelo["ESTADO"] != "GRADUADO"]

# Criar a variável EVASAO
df_modelo["EVASAO"] = df_modelo["ESTADO"].apply(lambda x: 1 if x in ["INATIVO", "CANCELADO"] else 0)

# Verificar distribuição da variável EVASAO
print(df_modelo["EVASAO"].value_counts())

EVASAO
1    1709
0     726
Name: count, dtype: int64


### Transformação variaveis Categoricas

In [17]:
# Transformação das variáveis categóricas usando one-hot encoding
colunas_categoricas = ['GENERO', 'ESTADO_CIVIL_ALUNOS', 'NACIONALIDADE', 'TIPO_ADMISSAO', 
                       'POLITICA_AFIRMATIVA', 'TIPO_ENSINO_MEDIO']
df_modelo = pd.get_dummies(df_modelo, columns=colunas_categoricas, drop_first=True)

# Exibir as primeiras linhas do dataframe transformado
print(df_modelo.head())

     IDADE  ANO_FORMATURA_ENSINO_MEDIO  CODIGO_CURSO  TERMO_ESTADO  CREDITOS  \
90      40                      2014.0      14102100        2005.1         4   
558     42                      2014.0      14102100        2006.1         4   
668     43                      2014.0      14102100        2008.2         4   
821     41                      2014.0      14102100        2006.1         4   
858     40                      2014.0      14102100        2007.2         4   

     HORAS  NOTA              ESTATUS   ESTADO  EVASAO  ...  \
90      60  -1.0             TRANCADO  INATIVO       1  ...   
558     60   0.0  REPROVADO_POR_FALTA  INATIVO       1  ...   
668     60   0.0  REPROVADO_POR_FALTA  INATIVO       1  ...   
821     60   7.2             APROVADO  INATIVO       1  ...   
858     60   1.3   REPROVADO_POR_NOTA  INATIVO       1  ...   

     POLITICA_AFIRMATIVA_L13  POLITICA_AFIRMATIVA_L14  POLITICA_AFIRMATIVA_L2  \
90                     False                    False      

### print(df_modelo.columns)

In [18]:
categorical_features = [
    'GENERO_MASCULINO',
    'ESTADO_CIVIL_ALUNOS_DESCONHECIDO', 'ESTADO_CIVIL_ALUNOS_DIVORCIADO', 
    'ESTADO_CIVIL_ALUNOS_SOLTEIRO', 'ESTADO_CIVIL_ALUNOS_VIUVO',
    'NACIONALIDADE_BRASILEIRA_POR_NATURALIZACAO', 'NACIONALIDADE_ESTRANGEIRA',
    'TIPO_ADMISSAO_DECISAO_JUDICIAL_ADM', 'TIPO_ADMISSAO_GRADUADO', 'TIPO_ADMISSAO_REOPCAO', 
    'TIPO_ADMISSAO_SISU', 'TIPO_ADMISSAO_TRANSFERENCIA', 'TIPO_ADMISSAO_VESTIBULAR',
    'POLITICA_AFIRMATIVA_BONUS', 'POLITICA_AFIRMATIVA_L1', 'POLITICA_AFIRMATIVA_L10', 
    'POLITICA_AFIRMATIVA_L13', 'POLITICA_AFIRMATIVA_L14', 'POLITICA_AFIRMATIVA_L2', 
    'POLITICA_AFIRMATIVA_L5', 'POLITICA_AFIRMATIVA_L6', 'POLITICA_AFIRMATIVA_L9',
    'TIPO_ENSINO_MEDIO_MAJORITARIAMENTE_PRIVADA', 'TIPO_ENSINO_MEDIO_MAJORITARIAMENTE_PUBLICA',
    'TIPO_ENSINO_MEDIO_PRIVADA', 'TIPO_ENSINO_MEDIO_PUBLICA'
]

# As colunas numéricas continuam as mesmas
numerical_features = ['IDADE', 'ANO_FORMATURA_ENSINO_MEDIO', 'CODIGO_CURSO', 'TERMO_ESTADO', 
                      'CREDITOS', 'HORAS', 'NOTA']

In [20]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Listando as variáveis categóricas e numéricas
categorical_features = ['GENERO', 'ESTADO_CIVIL_ALUNOS', 'NACIONALIDADE', 'TIPO_ADMISSAO', 
                        'POLITICA_AFIRMATIVA', 'TIPO_ENSINO_MEDIO', 'ESTATUS', 'ESTADO']
numerical_features = ['IDADE', 'ANO_FORMATURA_ENSINO_MEDIO', 'CODIGO_CURSO', 'TERMO_ESTADO', 
                      'CREDITOS', 'HORAS', 'NOTA']

# Verificar quais colunas realmente existem no dataframe
categorical_features = [col for col in categorical_features if col in df_modelo.columns]
numerical_features = [col for col in numerical_features if col in df_modelo.columns]

print(f"Variáveis categóricas encontradas: {categorical_features}")
print(f"Variáveis numéricas encontradas: {numerical_features}")

# Criar um pipeline para as variáveis categóricas
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Criar um pipeline para as variáveis numéricas (padronização)
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Criar um pré-processador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Aplicar o pré-processamento aos dados
X = preprocessor.fit_transform(df_modelo)

# Visualizar o resultado do pré-processamento
print("Formato do conjunto de dados transformado:", X.shape)

Variáveis categóricas encontradas: ['ESTATUS', 'ESTADO']
Variáveis numéricas encontradas: ['IDADE', 'ANO_FORMATURA_ENSINO_MEDIO', 'CODIGO_CURSO', 'TERMO_ESTADO', 'CREDITOS', 'HORAS', 'NOTA']
Formato do conjunto de dados transformado: (2435, 14)


### Criação Feature Store

In [21]:
import joblib

# Criar nomes de colunas após a transformação OneHotEncoder
cat_columns = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
num_columns = numerical_features

# Unindo os nomes das colunas transformadas
columns_final = list(num_columns) + list(cat_columns)

# Criando o DataFrame final com os dados transformados
df_feature_store = pd.DataFrame(X, columns=columns_final)

# Salvando a Feature Store como CSV e Parquet
df_feature_store.to_csv("feature_store.csv", index=False)
df_feature_store.to_parquet("feature_store.parquet", index=False)

# Salvando o pré-processador para reutilização futura
joblib.dump(preprocessor, "preprocessor.pkl")

# Exibir as primeiras linhas
print(df_feature_store.head())

      IDADE  ANO_FORMATURA_ENSINO_MEDIO  CODIGO_CURSO  TERMO_ESTADO  CREDITOS  \
0  1.756239                   -0.132636           0.0     -2.700396  0.039374   
1  2.066959                   -0.132636           0.0     -2.439828  0.039374   
2  2.222319                   -0.132636           0.0     -1.892635  0.039374   
3  1.911599                   -0.132636           0.0     -2.439828  0.039374   
4  1.756239                   -0.132636           0.0     -2.153203  0.039374   

      HORAS      NOTA  ESTATUS_APROVADO  ESTATUS_CANCELADO  \
0  0.039374 -1.515207               0.0                0.0   
1  0.039374 -1.250284               0.0                0.0   
2  0.039374 -1.250284               0.0                0.0   
3  0.039374  0.657161               1.0                0.0   
4  0.039374 -0.905884               0.0                0.0   

   ESTATUS_REPROVADO_POR_FALTA  ESTATUS_REPROVADO_POR_NOTA  ESTATUS_TRANCADO  \
0                          0.0                         0.0  

In [23]:
import numpy as np

# Vamos usar df_feature_store como base
df = df_feature_store.copy()

# Criando variável alvo 'EVASAO' aleatória para simular evasão
# 1 = não evadiu, 0 = evadiu
np.random.seed(42)  # para reprodutibilidade
df["EVASAO"] = np.random.randint(0, 2, size=len(df))

# Separando variáveis preditoras e alvo
X = df.drop(columns=["EVASAO"])
y = df["EVASAO"]

# Exibindo amostra para conferir
print(df[["EVASAO"]].value_counts())


EVASAO
0         1243
1         1192
Name: count, dtype: int64


### 1. Script para pipeline com salvamento de modelo (Apêndice A)

In [25]:
# ml_pipeline.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# Separar dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Criar pipeline com modelo Random Forest
pipeline_modelo = Pipeline([
    ('classificador', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Treinamento
pipeline_modelo.fit(X_train, y_train)

# Avaliação
y_pred = pipeline_modelo.predict(X_test)
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))

# Salvando o pipeline treinado
joblib.dump(pipeline_modelo, "modelo_random_forest.pkl")
print("Modelo salvo como 'modelo_random_forest.pkl'")


Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.56      0.58      0.57       249
           1       0.54      0.52      0.53       238

    accuracy                           0.55       487
   macro avg       0.55      0.55      0.55       487
weighted avg       0.55      0.55      0.55       487

Modelo salvo como 'modelo_random_forest.pkl'


### 2. Registro com DVC (Apêndice B) no terminal (cmd ou bash)

In [26]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import joblib

# Carregando a feature store e target
df = pd.read_csv("feature_store.csv")
df["EVASAO"] = [1 if i % 2 == 0 else 0 for i in range(len(df))]  # Simulando rótulo (ajuste conforme necessário)

X = df.drop("EVASAO", axis=1)
y = df["EVASAO"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Início do experimento MLflow
mlflow.set_experiment("ufcg-evasao-rf")  # nome do experimento

with mlflow.start_run():
    # Parâmetros do modelo
    n_estimators = 100
    max_depth = 5

    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)

    # Previsões
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    # Logs
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_metric("accuracy", acc)

    # Relatório
    print("Relatório de Classificação:")
    print(classification_report(y_test, y_pred))

    # Salvando o modelo via MLflow
    mlflow.sklearn.log_model(model, "modelo-rf")

    # Salvando localmente também
    joblib.dump(model, "modelo_random_forest.pkl")


2025/04/23 17:16:36 INFO mlflow.tracking.fluent: Experiment with name 'ufcg-evasao-rf' does not exist. Creating a new experiment.


Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.49      0.42      0.45       244
           1       0.49      0.56      0.53       243

    accuracy                           0.49       487
   macro avg       0.49      0.49      0.49       487
weighted avg       0.49      0.49      0.49       487



2025/04/23 17:17:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### A.3 Armazenamento da Feature Store

In [ ]:
pd.DataFrame(X_transformado, columns=numericas).to_csv("feature_store.csv", index=False)
pd.DataFrame(X_transformado, columns=numericas).to_parquet("feature_store.parquet")

### A.4 Registro de Experimentos com MLflow

O MLflow é utilizado para logar os artefatos, parâmetros e pipeline utilizado.

In [ ]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment("Evasao_Aluno_UFCG")
with mlflow.start_run(run_name="Preprocessamento e Feature Store"):
    mlflow.log_artifact("feature_store.csv")
    mlflow.log_artifact("feature_store.parquet")
    mlflow.log_artifact("preprocessador.pkl")
    mlflow.log_param("scaler", "StandardScaler")
    mlflow.sklearmfln.log_model(pipeline, artifact_path="modelo_pipeline")

### A.5 Versionamento com DVC

Com os arquivos salvos, o controle de versão é feito utilizando o DVC. Abaixo, comandos a serem executados no terminal:

In [ ]:
### Para verificar o status:

In [ ]:
dvc status
dvc dag